### Get all run and walk workouts. clean them and save them to csv files

In [1]:
import requests
import os
import sys
import csv
import json
import urllib3
import pandas as pd
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [6]:
pd.set_option('display.max_colwidth', None)

In [2]:
cd = os.path.abspath(os.getcwd())
with open(f'{cd}/personal.json') as f:
    data = json.load(f)

In [63]:
auth_url = "https://www.strava.com/oauth/token"
payload = data['payload']
res = requests.post(auth_url, data=payload, verify=False)
headers = {'Authorization': "Bearer " + res.json()['access_token']}

with open(f"{cd}/csvs/runs.csv", "w") as runs_file, open(f"{cd}/csvs/walks.csv", "w") as walks_file:
    for page in range(1,100):
        r = requests.get(f"https://www.strava.com/api/v3/athlete/activities", headers = headers, params={'per_page': 200, 'page': page})
        response = r.json()
        
        runs_writer = csv.writer(runs_file, delimiter=",")
        runs_writer.writerow(["id", "polyline"])
        
        walks_writer = csv.writer(walks_file, delimiter=",")
        walks_writer.writerow(["id", "polyline"])
        
        for activity in response:
            if activity['type'] == 'Run':
                polyline = activity["map"]["summary_polyline"]
                if polyline != 'polyline':
                    runs_writer.writerow([activity["id"], polyline])
            elif activity['type'] == 'Walk':
                polyline = activity["map"]["summary_polyline"]
                if polyline != 'polyline':
                    walks_writer.writerow([activity["id"], polyline])

In [103]:
with open('csvs/runs.csv') as r_f, open('csvs/walks.csv') as w_f:
    runs_list = pd.read_csv(r_f)
    walks_list = pd.read_csv(w_f)

In [104]:
# (408, 2) -> (274, 2)
walks_list_clean = walks_list[~(walks_list.polyline == 'polyline')]
walks_list_clean.dropna(subset=['polyline'], inplace=True)
# (277, 2) -> (185, 2)
runs_list_clean = runs_list[~(runs_list.polyline == 'polyline')]
runs_list_clean.dropna(subset=['polyline'], inplace=True)

In [107]:
with open('csvs/clean_runs.csv', 'w') as r_f, open('csvs/clean_walks.csv', 'w') as w_f:
    runs_list_clean.to_csv(r_f)
    walks_list_clean.to_csv(w_f)